# Code for determining b coefficients (amplitudes of excited state)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import cmath #for complex numbers

from scipy.spatial import distance_matrix

We set $\Gamma$ = 1 so we simply have the time in units of $\Gamma$. Also we set $k_e r_{ij} = k$ where $k$ is some contant  

In [2]:
epsilon_0 = 8.8541878188e-12        #F * m^-1 or C^2⋅kg^−1⋅m^−3⋅s^2
hbar = 1.054571817e-34              #J * s 

k_e = 8e9 #m^-1   from first excited state of hydrogen


dipole = 3.336e-30 #C*m    is cruly p correspondinc to 1 Debye 
dipole_vector_hat = np.array([0, 1, 0]) #i y retning  
dipole_vector = dipole * dipole_vector_hat

spont_decay = 1/(4 * np.pi * epsilon_0) * (4 * k_e**3 * dipole**2) / (3 * hbar)
print(spont_decay)

647479505715249.4


In [9]:
import Qchains
Qchains.ChainGenerator(5, showChain=False)

array([[-4,  0,  0],
       [-3,  0,  0],
       [-2,  0,  0],
       [-1,  0,  0],
       [ 0,  0,  0],
       [ 1,  0,  0],
       [ 2,  0,  0],
       [ 3,  0,  0],
       [ 4,  0,  0]])

In [4]:
points = np.array([
    [-4, 0, 0],
    [-3, 0, 0],
    [-2, 1, 0],
    [-1, 0, 0],
    [0, 0, 0],
    [1, 0, 0],
    [2, 0, 0],
    [3, 0, 0],
    [4, 0, 0]
])
#points = points/(1/k_e) #convert to units of 1/k_e
print('Number of particles', points.shape[0])

Number of particles 9


In [5]:
# x, y, z = points[:, 0], points[:, 1], points[:, 2]

# #interactive scatter plot
# fig = go.Figure(data=[go.Scatter3d(
#     x=x, y=y, z=z,
#     mode='markers',
#     marker=dict(size=5, color=z, colorscale='Viridis', opacity=0.8)
# )])

# # Customize layout
# fig.update_layout(
#     title="Geometry of the atoms",
#     scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
#     margin=dict(l=0, r=0, b=0, t=40)
# )

Calculating $r_{ij}$ 

In [6]:
#distance_matrix = distance_matrix(points, points) eventuelt for hurtigere 
r_ij_vector = points[:, np.newaxis] - points
r_ij = np.linalg.norm(r_ij_vector, axis=2, keepdims=True)
r_ij_hat = np.divide(r_ij_vector, r_ij, out=np.zeros_like(r_ij_vector, dtype=float), where=(r_ij != 0)) 

Calculate $F_{ji} = f_{ji} + i g_{ji}$

In [7]:
F = np.zeros(shape = (len(r_ij), len(r_ij)), dtype=complex)

for l in range(len(r_ij)): 
   dot_product = np.dot(r_ij_hat[l, :, :] , dipole_vector_hat)

   denom1 = (k_e *r_ij[l].flatten())
   denom2 = (k_e *r_ij[l].flatten())**2
   denom3 = (k_e *r_ij[l].flatten())**3
   safe_denom1 = np.where(denom1 == 0, np.nan, denom1)
   safe_denom2 = np.where(denom2 == 0, np.nan, denom2)
   safe_denom3 = np.where(denom3 == 0, np.nan, denom3)

   f_ji =  3/2 * (1 - (dot_product)**2) * np.sin(k_e *r_ij[l].flatten())/safe_denom1 \
      + 3/2 * (1 - 3 * (dot_product)**2) * (np.cos(k_e *r_ij[l].flatten())/safe_denom2 - np.sin(k_e *r_ij[l].flatten())/ safe_denom3) 
   g_ji = -3/2 * (1 - (dot_product)**2) * np.cos(k_e *r_ij[l].flatten())/safe_denom1 \
      + 3/2 * (1 - 3 * (dot_product)**2) * (np.sin(k_e *r_ij[l].flatten())/safe_denom2 + np.cos(k_e *r_ij[l].flatten())/ safe_denom3) 
   
   f_ji = np.nan_to_num(f_ji)
   g_ji = np.nan_to_num(g_ji)

   #Fill F
   F[:, l] = f_ji + 1j * g_ji 

Let $A = -\frac{\Gamma}{2} (I + F)$ and solve for eigenvalues and eigenvectors of A. Then $b(t) = ve^{\lambda t}$

In [8]:
I = np.eye(len(r_ij), dtype = complex)

A = - spont_decay/2 * (I + F)

A_eigenvalues, A_eigenvectors = np.linalg.eig(A)
print('Eigenvalues', A_eigenvalues, '\n')
print('Eigenvectors', A_eigenvectors)

Eigenvalues [-3.23739753e+14-24283.4375j     -3.23739753e+14 +3122.32421875j
 -3.23739753e+14+44110.94809926j -3.23739753e+14  -104.15581621j
 -3.23739753e+14-41645.49615598j -3.23739753e+14-16523.11602926j
 -3.23739753e+14+18875.11103282j -3.23739753e+14+19174.77390612j
 -3.23739753e+14 -2727.14157242j] 

Eigenvectors [[ 0.00814653+0.066891j   -0.18738377-0.00557018j  0.21383023+0.42168964j
   0.54906563+0.j         -0.02220564-0.02496452j  0.19070369+0.01938747j
   0.56313756+0.j          0.60603797+0.j          0.08544924+0.16056836j]
 [-0.00602788+0.00676869j  0.24603988-0.07995265j -0.14903998-0.14470488j
  -0.37911657-0.26114571j -0.07357678-0.03030245j  0.22693098+0.05699531j
   0.32256206+0.16090606j  0.29478804+0.09651131j -0.28362927+0.22697835j]
 [ 0.24326297-0.08292018j  0.01083237-0.08216541j  0.07683647-0.14657885j
  -0.33078153-0.20566407j  0.07493722-0.08561336j  0.01923984+0.1229055j
   0.10757401-0.2943937j   0.11558675-0.31443692j  0.83301369+0.j        ]
 [ 0.247173